In [1]:
#Imports
import math
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
from tslearn.utils import *
from tslearn.clustering import TimeSeriesKMeans
from contextlib import redirect_stdout

In [48]:
seed = 1
np.random.seed(seed)

#Get Companies
#companyStockFile = sys.argv[1]
#companiesCSV = pd.read_csv(companyStockFile,index_col = 0)

companiesCSV = pd.read_csv('./DataFrames/Stock data.csv',index_col = 0)
display(companiesCSV)

AMD CSCO INTC TXN

hello


,Year,Month,Day,AMZN,AMD,AAPL,CSCO,GOOGL,IBM,INTC,NVDA,TXN
0,2008,1,2,96.25,7.14,24.2913,20.6960,342.9379,73.8738,17.5161,30.4223,24.1292
1,2008,1,3,95.21,6.77,24.3026,20.8598,343.0080,74.0220,17.0463,30.1827,23.5996
2,2008,1,4,88.79,6.25,22.4474,20.3685,328.8288,71.3617,15.6643,27.6483,22.7717
3,2008,1,7,88.82,6.08,22.1470,20.3763,324.9500,70.5996,15.8094,24.7913,22.7419
4,2008,1,8,87.88,6.00,21.3503,19.8305,316.1562,68.8637,15.3810,25.3166,22.7493
...,...,...,...,...,...,...,...,...,...,...,...,...
2975,2019,10,25,1761.33,32.71,246.5800,46.9000,1264.3000,133.8590,56.1510,204.5400,119.6084
2976,2019,10,28,1777.08,33.69,249.0500,47.1700,1288.9800,134.3828,56.4493,206.7900,119.1022
2977,2019,10,29,1762.71,33.03,243.2900,47.2400,1260.6600,132.2579,56.0316,202.9200,119.3900
2978,2019,10,30,1779.99,33.13,243.2600,47.5600,1260.7000,133.6712,56.2902,203.0000,117.7900


In [41]:
#Split companies into their own data structures
#Format each company to a time series for use with tslearn
companyNames = []
companyStockPrices = []
formatted_time_series = []
numCompanies = len(companiesCSV.columns)-3

for company in range(0,numCompanies):
    companyNames.append(companiesCSV.columns.values[company+3])
    companyStockPrices.append(companiesCSV.filter(items=[companyNames[company]]))
    initialStockPrice = companyStockPrices[company].iloc[0,0]
    companyStockPrices[company] = pd.DataFrame([x - initialStockPrice for x in companyStockPrices[company].iloc[:,0]],columns=[companyNames[company]])
    companyStockPrices[company] = companyStockPrices[company].T
    companyStockPrices[company] = companyStockPrices[company].to_numpy()
    formatted_time_series.append(to_time_series(companyStockPrices[company]))

In [42]:
numClusters = 4
seed = 5
np.random.seed(seed)
formatted_dataset = to_time_series_dataset(formatted_time_series)
sz = formatted_dataset.shape[1]
km = TimeSeriesKMeans(n_clusters=numClusters,metric="euclidean",
                                  random_state=seed,n_init=3)   
clusterPredictions = km.fit_predict(formatted_dataset)

In [56]:
companyNameLines = ['','','','']
clusterTxt = open(r"clusters.txt","w")
for cluster in range(numClusters):
    figName = "Cluster" + str(cluster+1) + '.png'
    fig = plt.figure()#figsize=(20,10))
    #plt.tight_layout()
    ax = fig.add_axes([0,0,1,1])
    for j in range(0,len(formatted_dataset)):
        if(clusterPredictions[j] == cluster):
            companyNameLines[cluster] = companyNameLines[cluster] + companyNames[j] + ' '
            xx = formatted_dataset[j]
            ax.plot(xx.ravel(), alpha=.2, label=companyNames[j]) #"k-"
    ax.plot(km.cluster_centers_[cluster].ravel(), "r-", label='Cluster Center') 
    ax.set_title("Cluster " + str(cluster+1))
    lgd = ax.legend(bbox_to_anchor=(1, 1),fontsize = 'large')
    fig.savefig(figName,bbox_inches="tight")
    companyNameLines[cluster] = companyNameLines[cluster].rstrip() + '\n'
clusterTxt.writelines(companyNameLines)
clusterTxt.close()

C:\Users\aaron\Anaconda3\lib\site-packages\matplotlib\pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [54]:
clusterTxt.close()